# MMDetection

To develop an object detection model using mmdetection libraries which is based on PyTorch framework. It is improvised for better performance and flexible usage.

In [1]:
import os 
from os.path import exists, join, basename, splitext

## Installation

There are two versions of MMCV:

* mmcv: lite, without CUDA ops but all other features, similar to mmcv<1.0.0. It is useful when we do not need those CUDA ops.

* mmcv-full: comprehensive, with full features and various CUDA ops out of box. It takes longer time to build.

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# from google.colab import drive
# drive.flush_and_unmount()

In [3]:
mmdetection_dir = "/content/mmdetection"

In [4]:
# install dependencies: (use cu101 because colab has CUDA 10.1)
!pip install -U torch==1.5.1+cu101 torchvision==0.6.1+cu101 -f https://download.pytorch.org/whl/torch_stable.html
 
# install mmcv-full thus we could use CUDA operators
!pip install mmcv-full
 
# Install mmdetection
!rm -rf mmdetection
!git clone https://github.com/open-mmlab/mmdetection.git
%cd mmdetection
 
!pip install -e .
 
# install Pillow 7.0.0 back in order to avoid bug in colab
# !pip install Pillow==7.0.0

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 704.4MB 26kB/s 
     |████████████████████████████████| 6.7MB 59.3MB/s 
ERROR: torchtext 0.9.1 has requirement torch==1.8.1, but you'll have torch 1.5.1+cu101 which is incompatible.
  Found existing installation: torch 1.8.1+cu101
    Uninstalling torch-1.8.1+cu101:
      Successfully uninstalled torch-1.8.1+cu101
  Found existing installation: torchvision 0.9.1+cu101
    Uninstalling torchvision-0.9.1+cu101:
      Successfully uninstalled torchvision-0.9.1+cu101
     |████████████████████████████████| 266kB 12.9MB/s 
     |████████████████████████████████| 194kB 35.0MB/s 
  Created wheel for mmcv-full: filename=mmcv_full-1.3.1-cp37-cp37m-linux_x86_64.whl size=20891471 sha256=c25aa312c5c1fbad7efd618f8a7c7bcb23b8f164e39d050551a5fee180fe5978
  Stored in directory: /root/.cache/pip/wheels/75/40/57/0825abf01599edf55ba2a23444af249e79205dc0572786418d
Successfully built mmcv-full
Cloni

In [5]:
# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check MMDetection installation
import mmdet
print(mmdet.__version__)


# Check mmcv installation
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(get_compiling_cuda_version())
print(get_compiler_version())

1.5.1+cu101 True
2.11.0
11.0
GCC 7.5


In [6]:
%cd /content/mmdetection

/content/mmdetection


In [7]:
# config_file = "configs/pascal_voc/retinanet_r50_fpn_1x_voc0712.py"
config_file = '/content/mmdetection/configs/pascal_voc/faster_rcnn_r50_fpn_1x_voc0712.py'

# config_file = "configs/hrnet/cascade_rcnn_hrnetv2p_w32_20e_coco.py"
# config_file = "configs/hrnet/cascadedrcnn_rcnn__hrnet_r50_voc.py"

In [8]:
import os
config_fname = os.path.join(mmdetection_dir, config_file)

assert os.path.isfile(config_fname), '`{}` not exist'.format(config_fname)
config_fname

'/content/mmdetection/configs/pascal_voc/faster_rcnn_r50_fpn_1x_voc0712.py'

In [9]:
!cat {config_fname}

_base_ = [
    '../_base_/models/faster_rcnn_r50_fpn.py', '../_base_/datasets/voc0712.py',
    '../_base_/default_runtime.py'
]
model = dict(roi_head=dict(bbox_head=dict(num_classes=2)))
# optimizer
optimizer = dict(type='SGD', lr=0.001, momentum=0.9, weight_decay=0.0001)
optimizer_config = dict(grad_clip=None)
classes=('person','handbag')
# learning policy
# actual epoch = 3 * 3 = 9
lr_config = dict(policy='step', step=[4])
# runtime settings
# total_epochs = 4  # actual epoch = 4 * 3 = 12

dataset_type = 'VOCDataset'
data_root = r'content/data/VOCdevkit/VOC2007'
data = dict(
    train=dict(
        dataset=dict(
            type='VOCDataset',
            ann_file=data_root+'/Main/train.txt',
            img_prefix=data_root,
            classes=classes
            )),
    val=dict(
        type='VOCDataset',
        ann_file=data_root+'/Main/val.txt',
        img_prefix=data_root,
        classes=classes
        ),
    test=dict(
        type='VOCDataset',
        ann_file=data_root+

In [10]:
%cd {mmdetection_dir}

/content/mmdetection


### Installing dataset for model training

In [13]:
# !rm -rf /content/data

In [19]:
%cd /content/
!unzip /content/drive/MyDrive/Colab\ Notebooks/person_handbag_detection/data_V3.zip -d /content/

Streaming output truncated to the last 5000 lines.
  inflating: /content/data/VOCdevkit/VOC2007/JPEGImages/000000136316.jpg  
  inflating: /content/data/VOCdevkit/VOC2007/JPEGImages/000000136468.jpg  
  inflating: /content/data/VOCdevkit/VOC2007/JPEGImages/000000136563.jpg  
  inflating: /content/data/VOCdevkit/VOC2007/JPEGImages/000000136651.jpg  
  inflating: /content/data/VOCdevkit/VOC2007/JPEGImages/000000136663.jpg  
  inflating: /content/data/VOCdevkit/VOC2007/JPEGImages/000000136670.jpg  
  inflating: /content/data/VOCdevkit/VOC2007/JPEGImages/000000136683.jpg  
  inflating: /content/data/VOCdevkit/VOC2007/JPEGImages/000000136701.jpg  
  inflating: /content/data/VOCdevkit/VOC2007/JPEGImages/000000136836.jpg  
  inflating: /content/data/VOCdevkit/VOC2007/JPEGImages/000000136953.jpg  
  inflating: /content/data/VOCdevkit/VOC2007/JPEGImages/000000136979.jpg  
  inflating: /content/data/VOCdevkit/VOC2007/JPEGImages/000000137075.jpg  
  inflating: /content/data/VOCdevkit/VOC2007/JPEG

### Modify `voc.py`

parse data classes

In [20]:
import os
import glob
import pandas as pd
import xml.etree.ElementTree as ET

In [23]:
anno_path = "/content/data/VOCdevkit/VOC2007/Annotations"
voc_file = os.path.join(mmdetection_dir, "mmdet/datasets/voc.py")

assert os.path.isfile(voc_file), '`{}` not exist'.format(voc_file)
voc_file

'/content/mmdetection/mmdet/datasets/voc.py'

In [24]:
classes_names = []
xml_list = []
for xml_file in glob.glob(anno_path + "/*.xml"):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    for member in root.findall("object"):
        # print(member[0].text)
        classes_names.append(member[0].text)
classes_names = list(set(classes_names))
classes_names.sort()
classes_names
# len(classes_names)

['handbag', 'person']

In [25]:
import re

fname = voc_file
with open(fname) as f:
    s = f.read()
    s = re.sub('CLASSES = \(.*?\)',
               'CLASSES = ({})'.format(", ".join(["\'{}\'".format(name) for name in classes_names])), s, flags=re.S)
with open(fname, 'w') as f:
    f.write(s)
!cat {voc_file}

from collections import OrderedDict

from mmcv.utils import print_log

from mmdet.core import eval_map, eval_recalls
from .builder import DATASETS
from .xml_style import XMLDataset


@DATASETS.register_module()
class VOCDataset(XMLDataset):

    CLASSES = ('handbag', 'person')

    def __init__(self, **kwargs):
        super(VOCDataset, self).__init__(**kwargs)
        if 'VOC2007' in self.img_prefix:
            self.year = 2007
        elif 'VOC2012' in self.img_prefix:
            self.year = 2012
        else:
            raise ValueError('Cannot infer dataset year from img_prefix')

    def evaluate(self,
                 results,
                 metric='mAP',
                 logger=None,
                 proposal_nums=(100, 300, 1000),
                 iou_thr=0.5,
                 scale_ranges=None):
        """Evaluate in VOC protocol.

        Args:
            results (list[list | tuple]): Testing results of the dataset.
            metric (str | list[str]): Metrics to be e

make work directory

The below command will re-run the **mmdetection** package installing script so the changes to the voc.py file will be updated to the system python packages.

Since your data directory resides outside of the **mmdetection** directory, we have the following cell in the notebook which creates a symbolic link into the project data directory.

In [36]:
os.makedirs("dataset/VOCdevkit", exist_ok=True)
voc2007_dir = "dataset/VOC2007"
os.system("ln -s {} dataset/VOCdevkit".format(voc2007_dir))

256

In [35]:
WORK_DIR = "/content/work_directory/"
!mkdir /content/work_directory

mkdir: cannot create directory ‘/content/work_directory’: File exists


## Model Training

In [ ]:
# !rm -rf /content/work_directory
# !mkdir /content/work_directory

In [38]:
!python setup.py install

running install
running bdist_egg
running egg_info
writing mmdet.egg-info/PKG-INFO
writing dependency_links to mmdet.egg-info/dependency_links.txt
writing requirements to mmdet.egg-info/requires.txt
writing top-level names to mmdet.egg-info/top_level.txt
adding license file 'LICENSE' (matched pattern 'LICEN[CS]E*')
writing manifest file 'mmdet.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
copying mmdet/datasets/voc.py -> build/lib/mmdet/datasets
copying mmdet/core/evaluation/class_names.py -> build/lib/mmdet/core/evaluation
creating build/bdist.linux-x86_64/egg
creating build/bdist.linux-x86_64/egg/mmdet
copying build/lib/mmdet/version.py -> build/bdist.linux-x86_64/egg/mmdet
creating build/bdist.linux-x86_64/egg/mmdet/models
creating build/bdist.linux-x86_64/egg/mmdet/models/backbones
copying build/lib/mmdet/models/backbones/detectors_resnet.py -> build/bdist.linux-x86_64/egg/mmdet/models/backbones
copying build/lib/

In [39]:
%cd {mmdetection_dir}

!python tools/train.py '/content/mmdetection/configs/pascal_voc/faster_rcnn_r50_fpn_1x_voc0712.py' --work-dir {WORK_DIR}

/content/mmdetection
2021-04-24 09:21:14,116 - mmdet - INFO - Environment info:
------------------------------------------------------------
sys.platform: linux
Python: 3.7.10 (default, Feb 20 2021, 21:17:23) [GCC 7.5.0]
CUDA available: True
GPU 0: Tesla T4
CUDA_HOME: /usr/local/cuda
NVCC: Build cuda_11.0_bu.TC445_37.28845127_0
GCC: gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
PyTorch: 1.5.1+cu101
PyTorch compiling details: PyTorch built with:
  - GCC 7.3
  - C++ Version: 201402
  - Intel(R) Math Kernel Library Version 2019.0.5 Product Build 20190808 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v0.21.1 (Git Hash 7d2fd500bc78936d1d648ca713b901012f470dbc)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 10.1
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,code=sm_60;-gencode;arch=compute_61,code=sm_61;-gencode;arch=compute_70,code=sm_70;-ge

In [42]:
# !cp -r /content/work_directory/ /content/drive/MyDrive/Colab\ Notebooks/Faster_rcnn_table/3_dec/
!cp -r /content/work_directory/ /content/drive/MyDrive/Colab\ Notebooks/person_handbag_detection/24_apr/

In [43]:
checkpoint_file = os.path.join(mmdetection_dir, WORK_DIR, "latest.pth")
assert os.path.isfile(
    checkpoint_file), '`{}` not exist'.format(checkpoint_file)
checkpoint_file

'/content/work_directory/latest.pth'

## Analysis on Test Data

Get mAP on Test data

In [49]:
config_fname = '/content/mmdetection/configs/pascal_voc/faster_rcnn_r50_fpn_1x_voc0712.py'

In [52]:
# %cd {mmdetection_dir}
# !python tools/test.py {config_fname} {checkpoint_file} --eval mAP
!python tools/test.py {config_fname} {checkpoint_file} --out /content/test_results/out2.pkl --eval mAP

Use load_from_local loader
[>>] 370/370, 13.2 task/s, elapsed: 28s, ETA:     0s
writing results to /content/test_results/out2.pkl

---------------iou_thr: 0.5---------------

+---------+------+------+--------+-------+
| class   | gts  | dets | recall | ap    |
+---------+------+------+--------+-------+
| person  | 2305 | 9440 | 0.822  | 0.670 |
| handbag | 644  | 3144 | 0.526  | 0.298 |
+---------+------+------+--------+-------+
| mAP     |      |      |        | 0.484 |
+---------+------+------+--------+-------+
OrderedDict([('AP50', 0.484), ('mAP', 0.4839189648628235)])


Run inference on test image and save to a directory

In [59]:
!python tools/test.py {config_fname} {checkpoint_file} --show-dir /content/test_results/ --show-score-thr 0.70

Use load_from_local loader
[>>] 370/370, 5.7 task/s, elapsed: 65s, ETA:     0s

In [60]:
# !zip -r /content/file.zip /content/Folder_To_Zip
!zip -r /content/test_results.zip /content/test_results

updating: content/test_results/ (stored 0%)
updating: content/test_results/out2.pkl (deflated 28%)
  adding: content/test_results/JPEGImages/ (stored 0%)
  adding: content/test_results/JPEGImages/000000489785.jpg (deflated 0%)
  adding: content/test_results/JPEGImages/000000467151.jpg (deflated 1%)
  adding: content/test_results/JPEGImages/000000472484.jpg (deflated 0%)
  adding: content/test_results/JPEGImages/000000483254.jpg (deflated 0%)
  adding: content/test_results/JPEGImages/000000485123.jpg (deflated 0%)
  adding: content/test_results/JPEGImages/000000493173.jpg (deflated 1%)
  adding: content/test_results/JPEGImages/000000476344.jpg (deflated 2%)
  adding: content/test_results/JPEGImages/000000477774.jpg (deflated 1%)
  adding: content/test_results/JPEGImages/000000495626.jpg (deflated 0%)
  adding: content/test_results/JPEGImages/000000476514.jpg (deflated 0%)
  adding: content/test_results/JPEGImages/000000468263.jpg (deflated 0%)
  adding: content/test_results/JPEGImages/0

In [54]:
from mmdet.apis import inference_detector, init_detector, show_result_pyplot

# initialize the detector
model = init_detector(config_fname, checkpoint_file, device='cuda')

Use load_from_local loader


In [61]:
# Use the detector to do inference
img = '/content/fashion-2015-09-jaime-king-white-dress-sneakers-style-main.jpg'
result = inference_detector(model, img)

In [62]:
# Let's plot the result
show_result_pyplot(model, img, result, score_thr=0.70)

Output hidden; open in https://colab.research.google.com to view.

## Download the config file

In [65]:
from google.colab import files

files.download(config_fname)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Download checkpoint file and inferences.


In [69]:
files.download('/content/test_results.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
files.download(checkpoint_file)

In [70]:
!cp -r /content/test_results.zip /content/drive/MyDrive/Colab\ Notebooks/person_handbag_detection/24_apr/